In [1]:
# Importações necessárias
import pandas as pd
import numpy as np
from main import *  # Importa todas as funções criadas
import warnings
warnings.filterwarnings('ignore')

# Configurações para melhor visualização
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## Carregamento e Preparação dos Dados

In [ ]:
print("🔄 CARREGANDO DADOS PARA ANÁLISE DE MIGRAÇÃO")
print("="*60)

# TODO: Implementar funções específicas de extração
def extrair_tabela_gdxp(nome_tabela):
    # Implementar extração do GDXP
    pass

def extrair_tabela_aws(nome_tabela):
    # Implementar extração da AWS
    pass

# Exemplo de uso (substitua pelas suas funções)
# df_original = extrair_tabela_gdxp("nome_da_tabela")
# df_aws = extrair_tabela_aws("nome_da_tabela")

print(f"✅ Dados carregados com sucesso!")
print(f"   📊 Base Original (GDXP): {len(df_original):,} registros")
print(f"   📊 Base AWS: {len(df_aws):,} registros")
print(f"   📅 Período de análise: {df_original['data_coluna'].min()} a {df_original['data_coluna'].max()}")

## Análise Volumétrica Inicial

In [ ]:
print("\n" + "="*60)
print("📊 ANÁLISE VOLUMÉTRICA INICIAL")
print("="*60)

# Executar comparação volumétrica
resultado_volumetria = comparar_volumetria(df_original, df_aws)

print(f"📈 RESUMO VOLUMÉTRICO:")
print(f"   🔹 Base Original (GDXP): {resultado_volumetria['volumetria_original']:,} registros")
print(f"   🔹 Base AWS: {resultado_volumetria['volumetria_aws']:,} registros")
print(f"   🔹 Diferença: {resultado_volumetria['diferenca']:,} registros")

if resultado_volumetria['diferenca'] > 0:
    percentual_perda = (resultado_volumetria['diferenca'] / resultado_volumetria['volumetria_original']) * 100
    print(f"   ⚠️  Perda de dados: {percentual_perda:.2f}%")
elif resultado_volumetria['diferenca'] < 0:
    percentual_ganho = abs((resultado_volumetria['diferenca'] / resultado_volumetria['volumetria_original']) * 100)
    print(f"   📈 Aumento de dados: {percentual_ganho:.2f}%")
else:
    print(f"   ✅ Volumetrias idênticas!")


## Análise de Contratos Ausentes

In [ ]:
print("\n" + "="*60)
print("🔍 ANÁLISE DE CONTRATOS AUSENTES")
print("="*60)

# Identificar contratos ausentes
df_ausentes, num_ausentes, amostra_ausentes = contratos_ausentes(
    df_original, df_aws, 'coluna_contrato'
)

print(f"📋 CONTRATOS NÃO MIGRADOS:")
print(f"   🔹 Total de contratos ausentes: {num_ausentes:,}")
print(f"   🔹 Arquivo gerado: 'contratos_ausentes.csv'")

if num_ausentes > 0:
    percentual_ausentes = (num_ausentes / df_original['coluna_contrato'].nunique()) * 100
    print(f"   🔹 Percentual de contratos não migrados: {percentual_ausentes:.2f}%")
    
    print(f"\n📝 AMOSTRA DE CONTRATOS AUSENTES:")
    if not amostra_ausentes.empty:
        for idx, contrato in enumerate(amostra_ausentes['coluna_contrato'].head(), 1):
            print(f"   {idx}. {contrato}")
    
    if num_ausentes > 5:
        print(f"   ... e mais {num_ausentes - 5} contratos (ver arquivo CSV)")
else:
    print(f"   ✅ Todos os contratos foram migrados!")


## Análise Volumétrica Filtrada

In [ ]:
print("\n" + "="*60)
print("📊 ANÁLISE VOLUMÉTRICA - CONTRATOS COMUNS")
print("="*60)

# Comparar volumetria apenas dos contratos presentes em ambas as bases
resultado_filtrado = comparar_volumetria_filtrada(df_original, df_aws, 'coluna_contrato')

print(f"📈 VOLUMETRIA DOS CONTRATOS COMUNS:")
print(f"   🔹 Base Original (filtrada): {resultado_filtrado['volumetria_original_filtrada']:,} registros")
print(f"   🔹 Base AWS (filtrada): {resultado_filtrado['volumetria_aws_filtrada']:,} registros")
print(f"   🔹 Diferença: {resultado_filtrado['diferenca_filtrada']:,} registros")

if resultado_filtrado['diferenca_filtrada'] != 0:
    print(f"   ⚠️  Ainda existem diferenças volumétricas mesmo com contratos comuns")
else:
    print(f"   ✅ Volumetrias idênticas para contratos comuns!")


## Análise de Últimas Partições

In [ ]:
print("\n" + "="*60)
print("🎯 ANÁLISE DE ÚLTIMAS PARTIÇÕES")
print("="*60)

# Verificar se as volumetrias batem considerando apenas últimas partições
volumetrias_batem = comparar_ultimas_particoes(
    df_original, df_aws, 'coluna_contrato', 'coluna_particao'
)

print(f"🔍 COMPARAÇÃO DAS ÚLTIMAS PARTIÇÕES:")
if volumetrias_batem:
    print(f"   ✅ As volumetrias BATEM quando consideradas apenas as últimas partições!")
    print(f"   💡 Isso confirma que o problema são registros históricos duplicados na AWS")
else:
    print(f"   ❌ As volumetrias NÃO BATEM mesmo considerando apenas as últimas partições")
    print(f"   ⚠️  Pode haver outros problemas além da duplicação de partições")


## Comparação de Colunas

In [ ]:
print("\n" + "="*60)
print("🔬 ANÁLISE DE DIFERENÇAS POR COLUNA")
print("="*60)

# Definir colunas importantes para comparar
colunas_importantes = ['coluna1', 'coluna2', 'coluna3']  # Substitua pelas suas colunas

# Comparar valores das colunas
diferencas_colunas = comparar_colunas_por_contrato(
    df_original, df_aws, 'coluna_contrato', colunas_importantes
)

print(f"📊 DIFERENÇAS POR COLUNA:")
total_contratos_comuns = len(set(df_original['coluna_contrato'].unique()) & 
                            set(df_aws['coluna_contrato'].unique()))

for coluna, num_diferencas in diferencas_colunas.items():
    if num_diferencas > 0:
        percentual = (num_diferencas / total_contratos_comuns) * 100
        print(f"   ⚠️  {coluna}: {num_diferencas:,} contratos com diferenças ({percentual:.2f}%)")
    else:
        print(f"   ✅ {coluna}: Sem diferenças detectadas")

# Identificar colunas mais problemáticas
colunas_problematicas = [col for col, diff in diferencas_colunas.items() if diff > 0]
if colunas_problematicas:
    print(f"\n🚨 COLUNAS QUE REQUEREM ATENÇÃO:")
    for coluna in sorted(colunas_problematicas, key=lambda x: diferencas_colunas[x], reverse=True):
        print(f"   • {coluna}: {diferencas_colunas[coluna]:,} contratos afetados")


## Análises de Qualidade de Dados

In [ ]:
print("\n" + "="*60)
print("🔍 ANÁLISES DE QUALIDADE DE DADOS")
print("="*60)

# Executar análises integradas de qualidade
resultados_qualidade = executar_analises_qualidade(
    df_original, df_aws, 'coluna_contrato', 'coluna_particao'
)

# Os relatórios já são impressos pela função, mas você pode acessar os dados:
print(f"\n📋 RESUMO DAS ANÁLISES DE QUALIDADE:")
print(f"   🔹 Colunas com alertas de nulos: {len(resultados_qualidade['colunas_com_alerta_nulos'])}")
print(f"   🔹 Novos contratos duplicados: {resultados_qualidade['analise_duplicatas']['novos_duplicados']['quantidade']}")
print(f"   🔹 Contratos em múltiplas partições: {resultados_qualidade['estatisticas_particoes']['total_contratos_duplicados']}")
